In [1]:
import logging
from collections import namedtuple
import random

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [3]:
AUTOMATIC=False
rows=5
found=True

In [4]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [5]:
import copy
#I used an iterative loop with 2 steps (0 and 1), so the agent can see one step ahead
#in future re-calling itself, noticing if the propose move can give an advantage to
#his opponent
def hardcoded_agent(state:Nim)->Nimply:
    return iterative_loop(state, 0)

#I thought to this Evaluator as the best matrix to get some informations, without googling.
#Later i found out that it does something similar to the nim sum algorithm, even if
#I consider only some cases and I don't calculate all the possible moves
def iterative_loop(state: Nim, step:int) -> Nimply:
    global found
    #This evaluator counts how many rows have exactly the number of objects equal to the
    #current index of the evaluator
    Evaluator=[0]*rows*2
    for c in state.rows:
        if c!=0:            
            Evaluator[c]+=1

    if sum(Evaluator[2:])==1: #Final rush: aside one row, only 1s left in the others
        if Evaluator[1]%2==0:
            for r, c in enumerate(state.rows):
                if c>1:
                    return Nimply(r,c)
        else:
            for r, c in enumerate(state.rows):
                if c>1:
                    return Nimply(r,c-1)

    if sum(Evaluator)==3:    #[0,0,2,0,1,0,...]->[0,0,2,0,0,0,...]
        two=False
        objects=0
        for i, e in enumerate(Evaluator):
            if e==1:
                objects=i
            if e==2:
                two=True
        if two:
            for r, c in enumerate(state.rows):
                if c ==objects:
                    return Nimply(r,c)

    if sum(Evaluator)==2:  #[0,0,1,0,1,0,...]->[0,0,0,2,0,0,...]
        two=False
        objects_l=0
        objects_h=0
        for i, e in enumerate(Evaluator):
            if e==1 and not objects_l:
                objects_l=i
            if e==1 and objects_l:
                objects_h=i
            if e==2:
                two=True
        if not two:
            for r, c in enumerate(state.rows):
                if c ==objects_h:
                    return Nimply(r,c-objects_l)
    
    if sum(Evaluator)==1: #winning move  [0,1,0,0,...]->[0,0,0,0,...]
        for r, c in enumerate(state.rows):
                if c>0:
                    return Nimply(r,c)

    mask1=[0]*rows*2
    mask1[1]=2
    mask1[2]=mask1[3]=1#[0,2,1,1,0,...]
    mask2=[0]*rows*2
    mask2[2]=2
    mask2[1]=mask2[3]=1#[0,1,2,1,0,...]
    mask3=[0]*rows*2
    mask3[3]=2
    mask3[1]=mask3[2]=1#[0,1,1,2,0,...]

    if Evaluator==mask1:  #for all 3 masks, final configuration is [0,1,1,1,0,...]
        for r, c in enumerate(state.rows):
            if c==1:
                return Nimply(r,c)
    if Evaluator==mask2:
        for r, c in enumerate(state.rows):
            if c==2:
                return Nimply(r,c)
    if Evaluator==mask3:
        for r, c in enumerate(state.rows):
            if c==3:
                return Nimply(r,c)

    if sum(Evaluator[1:4])==2 and sum(Evaluator[4:])==1: #[0,1,0,1,0,1]->[0,1,1,1,0,0]
        objects_l=0
        objects_h=0
        for i, e in enumerate(Evaluator):
            if e==0 and not objects_l and i!=0:
                objects_l=i
            if e==1 and i>=4:
                objects_h=i
        for r, c in enumerate(state.rows):
            if c ==objects_h:
                return Nimply(r,c-objects_l)

    if Evaluator[1]==1 and Evaluator[2]==1 and Evaluator[3]==1 and sum(Evaluator[4:])==1:#[0,1,1,1,0,1,0,...]->[0,1,1,1,0,0,0,...]
        for r, c in enumerate(state.rows):
            if c >3:
                return Nimply(r,c)
 
    if step==0:
        i=0
        while found==True:
            i+=1
            row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
            num_objects = random.randint(1, state.rows[row])
            state2=copy.deepcopy(state)
            state2.nimming(Nimply(row, num_objects))
            ply=iterative_loop(state2,1)
            #if the opponent has already won, the agent won't find out an effective move to do, so it'd be stack 
            #in this while forever, so in a fixed iteration i force the agent to decide a random move anyway
            if i==100000: 
                return Nimply(row, num_objects)
        return Nimply(row, num_objects)

    roww = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objectss = random.randint(1, state.rows[roww])
    #if the function reach this points, it means that the agent has found a move in which the opponent doesn't win,
    #so it fixes the fount to false to let the same function at step 0 to exit the while and to return a move
    found=False
    return Nimply(roww, num_objectss)

In [6]:
def dumb_PCI(state: Nim) -> Nimply:
    """Pick always the minimum(maximum) possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], -m[1])))

In [7]:
#if you want to play manually against my agent, you are welcome :)
def manual(state:Nim)-> Nimply:
    while True:
        try:
            row=int(input("Enter the number of row(from 1 up):"))
            if row<=len(state._rows) and row>0 and state._rows[row-1]!=0:
                row-=1
                break
            else:
                print("No objects in this row. Try again.")
        except ValueError:
            print("Invalid input. Try again.")
    while True:
        try:
            num_objects=int(input("Enter the number of objects to take:"))
            if num_objects>0 and state._rows[row]-num_objects>=0:
                break
            else:
                print("You cannot take this number of objects, be serious. Try again.")
        except ValueError:
            print("Invalid input. Try again.")
    return Nimply(row,num_objects)

In [8]:
logging.getLogger().setLevel(logging.DEBUG)

if AUTOMATIC:
    strategy = (hardcoded_agent, dumb_PCI)
else:
    strategy = (hardcoded_agent, manual)

nim = Nim(rows)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    found=True
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9>
DEBUG:root:status: After player 0 -> <1 3 5 0 9>


You cannot take this number of objects, be serious. Try again.
You cannot take this number of objects, be serious. Try again.


DEBUG:root:status: After player 1 -> <0 3 5 0 9>
DEBUG:root:status: After player 0 -> <0 3 5 0 6>


No objects in this row. Try again.
No objects in this row. Try again.


DEBUG:root:status: After player 1 -> <0 0 5 0 6>
DEBUG:root:status: After player 0 -> <0 0 5 0 5>


Invalid input. Try again.
Invalid input. Try again.
Invalid input. Try again.
Invalid input. Try again.
Invalid input. Try again.
Invalid input. Try again.
